In [13]:
import json
import ast
import pandas as pd

In [14]:
# Load sorted and modified data. Modified to drop cycles in the tree

with open('data/mod_sorted_data.json') as f:
    data1 = json.load(f)

In [15]:
# Convert students to list

data={}
for key in data1.keys():
    data[key] = ast.literal_eval(data1[key])

In [21]:
def count_descendants(data,des_dict,remaining_nodes):
    node = remaining_nodes[-1]

    # Check if node in in the keys of data

    #if str(node) not in data.keys():
    #    des_dict[node] = 0
    #    remaining_nodes.pop()
    #    return
    des = 0
    children = data[str(node)]
    for child in children:

        # Check if child is the same as node
        #if str(child) == str(node):
        #    print(child)
        #    continue
        
        if str(child) not in data.keys():
            des_dict[str(child)] = 0
        elif des_dict[str(child)] == -1:
            remaining_nodes.append(child)
            return
        else:
            des += des_dict[str(child)]
    des_dict[str(node)] = des + len(data[str(node)])
    remaining_nodes.pop()

In [23]:
des_dict = {key : -1 for key in data.keys()}
remaining_nodes = list(data.keys())
#count = 0
while remaining_nodes != []:
    count_descendants(data,des_dict,remaining_nodes)
    #count += 1
#print(count)

In [24]:
with open("data/descendants.json", 'w') as json_file:
    json.dump(des_dict, json_file)

In [29]:
descendants_df = pd.DataFrame.from_dict(des_dict, orient='index').reset_index()
descendants_df.head()
descendants_df = descendants_df.rename(columns={'index': 'id', 0: 'descendants'})
descendants_df.head()
descendants_df['id'] = descendants_df['id'].astype(int)

In [30]:
descendants_df.to_csv("data/descendants.csv", index=False)


In [31]:
data = pd.read_csv("data/data_mod.csv")

In [32]:
merged_df = data.merge(descendants_df, on='id', how='left')


In [33]:
merged_df.to_csv("data/data_des.csv", index=False)